In [1]:
from koselleck import *

In [2]:
fieldd=get_fields()

In [7]:
dfmodelsdec=get_pathdf_models_bydecade()
row=dfmodelsdec.sample(n=1).iloc[0]
m=load_model_row(row)
row.period, row.run, m

('1700-1710', 'run_21', <gensim.models.word2vec.Word2Vec at 0x7fc68383b710>)

In [6]:
round(to_dist(m, ['virtue','value','rock','door','doors']).sort_values('door'),6)

,rock,door,virtue,doors,value
doors,-0.467526,-4.703270,1.097239,NaN,1.546248
rock,NaN,-0.955930,0.792549,-0.467526,0.619571
virtue,0.792549,1.150751,NaN,1.097239,-1.330347
value,0.619571,2.250714,-1.330347,1.546248,NaN
door,-0.955930,NaN,1.150751,-4.703270,2.250714


In [ ]:
dfdist=to_dist(m, get_valid_words(), z=0)
dfdist

In [ ]:
dfdist.sort_values('door')['door'].head(25)

In [131]:
g=to_semnet_from_dist(dfdist)
g

100%|██████████| 5171/5171 [00:02<00:00, 1725.82it/s]


In [132]:
g.order(), g.size()

(5171, 41812)

In [133]:
list(g.neighbors('virtue'))

['void',
 'benevolence',
 'piety',
 'innocence',
 'probity',
 'wisdom',
 'temperance',
 'folly',
 'chastity',
 'humanity',
 'greatness']

In [134]:
clust=pd.Series(nx.clustering(g)).sort_values()

sake          0.000000
omen          0.000000
tome          0.000000
admonition    0.015152
quack         0.015152
                ...   
mariner       0.727273
march         0.745455
triangle      0.763636
ascension     0.822222
berry         0.888889
Length: 5171, dtype: float64

In [137]:
measure_ambiguity(m,words=get_valid_words())

100%|██████████| 5171/5171 [00:03<00:00, 1465.89it/s]


sake          1.000000
omen          1.000000
tome          1.000000
admonition    0.984848
quack         0.984848
                ...   
mariner       0.272727
march         0.254545
triangle      0.236364
ascension     0.177778
berry         0.111111
Length: 5171, dtype: float64

In [167]:
def compute_ambiguity_across_models(pathdf,words=None,num_proc=1,**attrs):
    #gby=[c for c in pathdf.columns if not c.startswith('path')]
    gby=['corpus','period','run']
    return pmap_groups(
        do_compute_ambiguity_across_models,
        pathdf.groupby(gby),
        num_proc=num_proc,
        kwargs=dict(words=words),
        desc='Computing key vectors across model runs'
    )
def do_compute_ambiguity_across_models(df,words=None,**y):
    l=[]
#     for i,row in tqdm(df.iterrows(), total=len(df)):
    for i,row in df.iterrows():
        m=load_model_row(row)
        amb=measure_ambiguity(m,words=words,**y)
        ambr=amb.rank()
        ambrz=(ambr - ambr.mean()) / ambr.std()
        l+=[amb]
    o=pd.DataFrame(l).mean()
    odf=pd.DataFrame([o])
    return odf

In [168]:
# gby=['corpus','period']
# for i,gdf in dfmodelsdec.groupby(gby): pass
# do_compute_ambiguity_across_models(gdf, words=get_valid_words())

In [170]:
ambdf=compute_ambiguity_across_models(dfmodelsdec, words=get_valid_words(), num_proc=6)
ambdf

Computing key vectors across model runs [x6]: 100%|██████████| 433/433 [19:39<00:00,  2.72s/it]


bath      step     stile      mint      fore  \
corpus period    run                                                        
bpo    1700-1710 run_01  1.000000  0.981818  0.981818  0.977778  0.977778   
                 run_02  1.000000  0.755556  0.733333  0.909091  0.955556   
                 run_03  0.933333  0.888889  0.933333  0.911111  1.000000   
                 run_04  0.933333  0.977778  0.954545  0.945455  1.000000   
                 run_05  1.000000  0.933333  0.872727  0.844444  1.000000   
...                           ...       ...       ...       ...       ...   
       1870-1880 run_05  0.933333  0.927273  0.600000  0.727273  0.911111   
                 run_06  0.911111  0.911111  0.780952  0.688889  0.800000   
                 run_07  0.866667  0.872727  0.636364  0.644444  0.911111   
       1880-1890 run_01  0.854545  0.909091  0.784314  0.688889  0.822222   
       1890-1900 run_01  0.854545  0.836364  0.575758  0.742424  0.818182   

                              age   quality      seal   admirer      seat  \
corpus period    run                                                        
bpo    1700-1710 run_01  0.977778  0.977778  0.977778  0.969697  0.963636   
                 run_02  0.866667  0.933333  0.844444  0.884615  0.709091   
                 run_03  0.888889  0.822222  0.933333  0.911111  0.955556   
                 run_04  0.777778  0.866667  0.863636  0.854545  0.654545   
                 run_05  0.800000  0.955556  0.800000  0.838095  0.733333   
...                           ...       ...       ...       ...       ...   
       1870-1880 run_05  0.618182  0.813187  0.897436  0.858974  0.714286   
                 run_06  0.622222  0.791209  0.909091  0.890909  0.717949   
                 run_07  0.622222  0.782051  0.897436  0.872727  0.780220   
       1880-1890 run_01  0.600000  0.679487  0.871795  0.803030  0.833333   
       1890-1900 run_01  0.466667  0.830882  0.836364  0.800000  0.705128   

                         ...     radio   massage  underwear   booklet  \
corpus period    run     ...                                            
bpo    1700-1710 run_01  ...       NaN       NaN        NaN       NaN   
                 run_02  ...       NaN       NaN        NaN       NaN   
                 run_03  ...       NaN       NaN        NaN       NaN   
                 run_04  ...       NaN       NaN        NaN       NaN   
                 run_05  ...       NaN       NaN        NaN       NaN   
...                      ...       ...       ...        ...       ...   
       1870-1880 run_05  ...       NaN       NaN        NaN       NaN   
                 run_06  ...       NaN       NaN        NaN       NaN   
                 run_07  ...       NaN       NaN        NaN       NaN   
       1880-1890 run_01  ...  0.888889  0.755556   0.696970  0.628205   
       1890-1900 run_01  ...  0.844444  0.844444   0.659341  0.763636   

                          cycling  aeroplane  unemployment  automobile  \
corpus period    run                                                     
bpo    1700-1710 run_01       NaN        NaN           NaN         NaN   
                 run_02       NaN        NaN           NaN         NaN   
                 run_03       NaN        NaN           NaN         NaN   
                 run_04       NaN        NaN           NaN         NaN   
                 run_05       NaN        NaN           NaN         NaN   
...                           ...        ...           ...         ...   
       1870-1880 run_05       NaN        NaN           NaN         NaN   
                 run_06       NaN        NaN           NaN         NaN   
                 run_07       NaN        NaN           NaN         NaN   
       1880-1890 run_01  0.564103   0.400000           NaN         NaN   
       1890-1900 run_01  0.549451   0.777778      0.836364    0.709091   

                         wireless   chiffon  
corpus period    run                         
bpo    1700-1710 run

In [172]:
ambdf.mean().sort_values()

march           0.338448
june            0.342443
august          0.345725
july            0.348447
constipation    0.411138
                  ...   
outburst        0.899182
presence        0.908123
arc             0.911377
planning        0.915859
stead           0.921570
Length: 5999, dtype: float64

In [173]:
ambdf.to_csv('data/data.ambiguity.runs.csv')

In [186]:
odf=ambdf.groupby('period').mean()
odf.index=[int(p[:4]) for p in odf.index]
odf=odf.T

In [187]:
ambdf.mean().sort_values()

march           0.338448
june            0.342443
august          0.345725
july            0.348447
constipation    0.411138
                  ...   
outburst        0.899182
presence        0.908123
arc             0.911377
planning        0.915859
stead           0.921570
Length: 5999, dtype: float64

In [193]:
to_z(odf)

100%|██████████| 20/20 [00:00<00:00, 1294.24it/s]


,1700,1710,1720,1730,1740,1750,1760,1770,1780,1790,1800,1810,1820,1830,1840,1850,1860,1870,1880,1890
bath,1.963315,0.930048,-0.380786,0.622211,0.468572,0.092130,-0.625146,1.160846,-1.085181,-1.986439,0.424064,0.602090,1.108912,1.410990,1.884339,1.991622,1.123162,1.442716,0.949148,0.926815
step,1.268047,1.155317,-0.366609,0.292503,-0.028536,0.759894,1.567814,1.219187,1.628019,1.199844,0.794112,1.345394,1.411459,1.352851,1.446417,0.902642,0.787814,1.458300,1.458940,0.758033
stile,1.400524,-0.517620,0.565658,-1.185496,-0.274898,0.334612,-1.307346,-1.583659,-1.086881,-1.398833,-1.972642,-1.095650,1.352080,1.015284,0.084646,0.497216,-0.030629,-1.005765,0.292749,-1.661176
mint,0.986120,-0.094163,0.856445,-0.850705,0.615954,0.229243,0.975771,-0.068881,0.580422,0.688396,-0.226947,-1.599311,-0.615292,1.000761,0.951360,1.008849,0.103969,-0.700935,-0.599109,-0.114008
fore,1.952831,1.354035,0.958317,-0.076544,0.356928,-2.774782,-2.292400,-1.474880,-0.002642,-0.179935,0.876439,1.063194,0.193362,-0.464485,-1.295067,0.432516,0.034345,0.842149,0.647049,0.589251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
aeroplane,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.299117,0.214180
unemployment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.758033
automobile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.423441
wireless,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.817266


In [ ]:
odf